<a href="https://colab.research.google.com/github/jbloewencolon/Psychedelic-Trip-Generator/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gensim
import re
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.base import BaseEstimator, TransformerMixin

In [58]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/processed.csv')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76450 entries, 0 to 76449
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             76448 non-null  object 
 1   drug              76447 non-null  object 
 2   dosage            73699 non-null  object 
 3   delivery          74213 non-null  object 
 4   weight            76449 non-null  float64
 5   year              76449 non-null  float64
 6   gender            76446 non-null  object 
 7   report            76439 non-null  object 
 8   processed_report  76438 non-null  object 
 9   mixed             76448 non-null  float64
 10  drug_category     76448 non-null  object 
dtypes: float64(3), object(8)
memory usage: 6.4+ MB


In [65]:
# Fill missing values with 'unknown' in 'drug_category' column
df[['drug', 'dosage', 'delivery']] = df[['drug', 'dosage', 'delivery']].fillna('unknown')

# Convert 'year' and 'mixed' columns to integers
df[['year', 'mixed']] = df[['year', 'mixed']].astype(int)

df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76438 entries, 0 to 76449
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             76438 non-null  object 
 1   drug              76438 non-null  object 
 2   dosage            76438 non-null  object 
 3   delivery          76438 non-null  object 
 4   weight            76438 non-null  float64
 5   year              76438 non-null  int64  
 6   gender            76438 non-null  object 
 7   report            76438 non-null  object 
 8   processed_report  76438 non-null  object 
 9   mixed             76438 non-null  int64  
 10  drug_category     76438 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 7.0+ MB


In [66]:
df.head()

,title,drug,dosage,delivery,weight,year,gender,report,processed_report,mixed,drug_category
0,Ode to Joy,mdma,1.5 tablets,oral,185.0,2000,male,My friend had some experience with X and had t...,friend experi x told one day said come across ...,0,mdma
1,Make Sure the Music's Not Too Complex,cannabis,unknown,smoked,152.0,1999,not specified,This was the first experience that either my f...,first experi either friend salvia housem check...,0,cannabis
2,After Hours,mdma,160 mg,oral,150.0,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,mdma
3,After Hours,mdma,100 mg,oral,150.0,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,mdma
4,After Hours,mdma,50 mg,insufflated,150.0,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,mdma


In [67]:
df['drug_category'].value_counts()

pharmaceutical           10125
cannabis                  9536
stimulant                 5780
mushrooms                 4055
botanical                 3814
opioid                    3795
mdma                      3560
alcohol                   3368
hallucinogen              3128
lsd                       3064
salvia                    2831
2c                        2812
other                     2746
dissociative              2624
entheogen                 2546
DMT                       2362
entactogen                1754
nootropic                 1653
5-meo                     1290
ketamine                  1233
unknown                    923
mescaline                  726
depressant                 686
synthetic cannabinoid      451
anxiolytic                 448
oneirogen                  307
ayahuasca                  229
phencyclidine              216
antidepressant             167
3-MeO                      131
ibogaine                    78
Name: drug_category, dtype: int64

In [68]:
import pandas as pd

# Define the mapping from old categories to new ones
category_mapping = {
    'pharmaceutical': 'Pharmaceutical',
    'cannabis': 'Cannabinoid',
    'stimulant': 'Stimulant',
    'mushrooms': 'Psychedelic',
    'botanical': 'Other',
    'opioid': 'Opioid',
    'mdma': 'Entactogen/Empathogen',
    'alcohol': 'Depressant',
    'hallucinogen': 'Psychedelic',
    'lsd': 'Psychedelic',
    'salvia': 'Psychedelic',
    '2c': 'Psychedelic',
    'other': 'Other',
    'dissociative': 'Dissociative',
    'entheogen': 'Entheogen',
    'DMT': 'Psychedelic',
    'entactogen': 'Entactogen/Empathogen',
    'nootropic': 'Other',
    '5-meo': 'Psychedelic',
    'ketamine': 'Dissociative',
    'unknown': 'Other',
    'mescaline': 'Psychedelic',
    'depressant': 'Depressant',
    'synthetic cannabinoid': 'Cannabinoid',
    'anxiolytic': 'Pharmaceutical',
    'oneirogen': 'Other',
    'ayahuasca': 'Psychedelic',
    'phencyclidine': 'Dissociative',
    'antidepressant': 'Pharmaceutical',
    '3-MeO': 'Other',
    'ibogaine': 'Entheogen'
}

# Apply the mapping to the 'drug_category' column and create a new column 'grouped_drug_category'
df['drug_category'] = df['drug_category'].map(category_mapping)

df['drug_category'].value_counts()

Psychedelic              20497
Pharmaceutical           10740
Cannabinoid               9987
Other                     9574
Stimulant                 5780
Entactogen/Empathogen     5314
Dissociative              4073
Depressant                4054
Opioid                    3795
Entheogen                 2624
Name: drug_category, dtype: int64

In [75]:
df.head()

,title,drug,dosage,delivery,weight,year,gender,report,processed_report,mixed,drug_category
0,Ode to Joy,mdma,1.5 tablets,oral,185.0,2000,male,My friend had some experience with X and had t...,friend experi x told one day said come across ...,0,Entactogen/Empathogen
1,Make Sure the Music's Not Too Complex,cannabis,unknown,smoked,152.0,1999,not specified,This was the first experience that either my f...,first experi either friend salvia housem check...,0,Cannabinoid
2,After Hours,mdma,160 mg,oral,150.0,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,Entactogen/Empathogen
3,After Hours,mdma,100 mg,oral,150.0,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,Entactogen/Empathogen
4,After Hours,mdma,50 mg,insufflated,150.0,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,Entactogen/Empathogen


In [69]:
# Tokenize the reports
tokenized_reports = df['processed_report'].apply(simple_preprocess)

# Train a Word2Vec model
word2vec = Word2Vec(sentences=tokenized_reports, vector_size=100, window=5, min_count=5, workers=4)

# Function to convert a report into a vector
def report_to_vector(report):
    report_vec = []
    numw = 0
    for word in report:
        try:
            if numw == 0:
                report_vec = word2vec.wv[word] # Use word2vec.wv[word] instead of model[word]
            else:
                report_vec = np.add(report_vec, word2vec.wv[word]) # Use word2vec.wv[word] instead of model[word]
            numw+=1
        except:
            pass

    return np.asarray(report_vec) / numw if numw != 0 else np.zeros(word2vec.vector_size)

# Convert reports into vectors
X = df['processed_report'].apply(report_to_vector).tolist()

In [70]:
class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([report_to_vector(report) for report in X])

# Instantiate Word2Vec model (replace with your model parameters and training data)
model = gensim.models.Word2Vec(sentences=tokenized_reports, vector_size=100, window=5, min_count=1, workers=4)

# Define feature extractor
vectorizer = Word2VecVectorizer(model)

In [80]:
# Create the GridSearchCV instance
grid_search = GridSearchCV(model, param_grid, cv=3)

# Tokenize the reports
tokenized_reports = df['processed_report'].apply(simple_preprocess)

# Train the Word2Vec model and perform grid search
grid_search.fit(tokenized_reports)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

TypeError: ignored

In [71]:
# Define LDA model
lda = LDA(n_components=10)

# Define label encoder
encoder = LabelEncoder()

# Fit and transform the processed reports
X = vectorizer.fit_transform(df['processed_report'])
y = encoder.fit_transform(df['drug_category'])

# Fit LDA model
lda.fit(X)

# Function to print the top words for each topic
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword_bound = bound / word_cnt


In [74]:
# Get the vocabulary from the Word2Vec model
vocab = list(word2vec.wv.index_to_key)

# Print the top words per topic
print_top_words(lda, vocab, 20)


Topic #0:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #1:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #2:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #3:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #4:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #5:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #6:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #7:
differ got thing smoke think seem thought first drug took decid even see almost good want come didnt littl bodi

Topic #8:
diffe

In [79]:
# Define the parameter grid
param_grid = {
    'n_components': [5, 10, 15],  # Number of topics
    'learning_method': ['batch', 'online'],  # Learning method
    'doc_topic_prior': [0.1, 0.5, 1.0],  # Alpha parameter
    'topic_word_prior': [0.01, 0.1, 1.0]  # Beta parameter
}

# Create the GridSearchCV object
grid_search = GridSearchCV(lda, param_grid, cv=3)

# Fit the grid search to the data
grid_search.fit(X)

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword_bound = bound / word_cnt
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword_bound = bound / word_cnt
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword_bound = bound / word_cnt
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword_bound = bound / word_cnt
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword_bound = bound / word_cnt
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_lda.py:889: RuntimeWarning: divide by zero encountered in double_scalars
  perword

KeyboardInterrupt: ignored

In [ ]:
# Define oversampler
oversampler = SMOTE()

# Compute class frequencies in 'mixed' column
class_freq = df['mixed'].value_counts()

# Define class weights based on frequencies
class_weights = {value: 10 if value == '0' else 1 for value in class_freq.index}

# Encode the labels in y
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Set up the XGBoost parameters
params = {
    'objective': 'multi:softmax',
    'num_class': 3
}

# Define classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight=class_weights),
    'Random Forest': RandomForestClassifier(class_weight=class_weights, n_estimators=100, random_state=42),
    'XGB': XGBClassifier(eval_metric='mlogloss')
}

# Define pipelines
pipelines = {
    name: make_pipeline(oversampler, model)
    for name, model in classifiers.items()
}

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit and evaluate Logistic Regression
print('\nTraining Logistic Regression...')
pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('\nLogistic Regression:')
print(classification_report(y_test, y_pred_lr, target_names=encoder.classes_, zero_division=1))

In [ ]:
# Fit and evaluate Random Forest
print('\nTraining Random Forest...')
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print('\nRandom Forest:')
print(classification_report(y_test, y_pred_rf, target_names=encoder.classes_, zero_division=1))

In [ ]:
# Fit and evaluate XGBoost with the pipeline
print('\nTraining XGB through pipeline...')
pipeline_xgb.fit(X_train, y_train, xgbclassifier__sample_weight=weights_train)
y_pred_xgb = pipeline_xgb.predict(X_test)
print('\nXGBoost (through pipeline):')
print(classification_report(y_test, y_pred_xgb, target_names=encoder.classes_, zero_division=1))

In [ ]:
# Define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# Create a GridSearchCV object
grid_search = GridSearchCV(lr, param_grid, cv=5)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(grid_search.best_params_)
print(grid_search.best_score_)
